![Aircraft](Images/aircraft.jpg)

# Aviation Industry analysis and insights

## Overview

In response to your company's strategic decision to diversify its portfolio, this report focuses on evaluating the aviation industry—specifically, the operational risks associated with different aircraft types. Using historical data on aviation accidents from the National Transportation Safety Board (NTSB), the objective is to identify the lowest-risk aircraft models to guide future investments in both commercial and private aviation.

## Business Understanding

As your company enters the aviation sector, minimizing risk is a top priority. The core business problem revolves around identifying which aircraft are most likely to offer safe and reliable service. This includes understanding patterns in aircraft accidents, assessing factors that contribute to safety, and ultimately recommending aircraft models that align with the company’s risk tolerance and operational goals.

## Data Understanding

The analysis leverages the NTSB aviation accident dataset, which includes records of civil aviation accidents from 1962 to 2023. To ensure the reliability and relevance of the insights, we first explored the structure and quality of the dataset. This stage involves understanding the key variables—such as aircraft type, number of fatalities, purpose of flight, and accident dates—that inform our assessment of risk across aircraft models.

Let's explore the structure of the dataset


In [36]:
# Essential imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Display settings
pd.set_option('display.max_columns', 100)
sns.set(style="whitegrid")

# importing dataset as df
df = pd.read_csv('Data/AviationData.csv', encoding="cp1252")

/var/folders/z5/_nf3rthd45d92_2bgsgb_thc0000gn/T/ipykernel_12596/3551501658.py:13: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/AviationData.csv', encoding="cp1252")


#### let's gather some basic info about the dataset

In [56]:
# Basic info
df.info(verbose=True, show_counts=True) #displaying all table structure

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                88889 non-null  object        
 1   Investigation.Type      88889 non-null  object        
 2   Accident.Number         88889 non-null  object        
 3   Event.Date              88889 non-null  datetime64[ns]
 4   Location                88837 non-null  object        
 5   Country                 88663 non-null  object        
 6   Latitude                34382 non-null  object        
 7   Longitude               34373 non-null  object        
 8   Airport.Code            50132 non-null  object        
 9   Airport.Name            52704 non-null  object        
 10  Injury.Severity         87889 non-null  object        
 11  Aircraft.damage         85695 non-null  object        
 12  Aircraft.Category       32287 non-null  object

In [57]:
df.describe()

,Event.Date,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,88889,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1999-09-17 17:13:39.354475904,1.146585,0.647855,0.279881,0.357061,5.325440
min,1948-10-24 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1989-01-15 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1998-07-18 00:00:00,1.000000,0.000000,0.000000,0.000000,1.000000
75%,2009-07-01 00:00:00,1.000000,0.000000,0.000000,0.000000,2.000000
max,2022-12-29 00:00:00,8.000000,349.000000,161.000000,380.000000,699.000000
std,NaN,0.446510,5.485960,1.544084,2.235625,27.913634


By observing, 25% 50% and 75% we can understand that something is wrong with those results, we will need some data cleaning process later

In [59]:
df['Total.Fatal.Injuries'].dtype

dtype('float64')

In [39]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,NC6404,Stinson,108-3,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N5069P,Piper,PA24-180,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),Destroyed,NaN,N5142R,Cessna,172M,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,N1168J,Rockwell,112,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),Destroyed,NaN,N15NY,Cessna,501,No,NaN,NaN,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [40]:
# Check nulls
df.isnull().sum().sort_values(ascending=False)

Schedule                  76307
Air.carrier               72241
FAR.Description           56866
Aircraft.Category         56602
Longitude                 54516
Latitude                  54507
Airport.Code              38757
Airport.Name              36185
Broad.phase.of.flight     27165
Publication.Date          13771
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Fatal.Injuries      11401
Engine.Type                7096
Report.Status              6384
Purpose.of.flight          6192
Number.of.Engines          6084
Total.Uninjured            5912
Weather.Condition          4492
Aircraft.damage            3194
Registration.Number        1382
Injury.Severity            1000
Country                     226
Amateur.Built               102
Model                        92
Make                         63
Location                     52
Investigation.Type            0
Event.Date                    0
Accident.Number               0
Event.Id                      0
dtype: i

Some of those columns have close to no values ( they are not really useful for our analysis )

In [50]:
# Value counts for key variables
df['Make'].value_counts().head(10)


Make
Cessna     22227
Piper      12029
CESSNA      4922
Beech       4330
PIPER       2841
Bell        2134
Boeing      1594
BOEING      1151
Grumman     1094
Mooney      1092
Name: count, dtype: int64

In [49]:
df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [51]:
df['Injury.Severity'].value_counts()

Injury.Severity
Non-Fatal     67357
Fatal(1)       6167
Fatal          5262
Fatal(2)       3711
Incident       2219
              ...  
Fatal(270)        1
Fatal(60)         1
Fatal(43)         1
Fatal(143)        1
Fatal(230)        1
Name: count, Length: 109, dtype: int64

In [77]:
#show the shape of the dataset
print(f"We are working a {df.shape[1]} columns dataset, it has {df.shape[0]} rows of record")

We are working a 31 columns dataset, it has 88889 rows of record


In [70]:
# Convert date to be sure of our data
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

In [ ]:
# Quick summary to support data understanding
print("This dataset covers events from ", df['Event.Date'].min().date(), "to", df['Event.Date'].max().date())
print(f"{df['Make'].nunique()} aircraft makes were being recorded")
print(f"For a total of {df.shape[0]} incidents")

This dataset covers events from  1948-10-24 to 2022-12-29
8237 aircraft makes were being recorded
For a total of 88889 incidents


### For our analysis, a few specific colums are required, Here is a breakdown
#### 🛫 Aircraft Identification
- Make — Manufacturer (e.g., Boeing, Cessna)
- Model — Aircraft model (e.g., 737-800)
- Aircraft.Category — E.g., airplane, helicopter, etc.
- Number.of.Engines
- Engine.Type — E.g., turbojet, turboprop, piston
- Amateur.Built — Indicates experimental aircraft (higher risk)

#### 📅 Event Context
- Event.Date — Needed to analyze trends and recent risks
- Location or Country — Optional, for filtering domestic vs. international
- Weather.Condition — Helps see if crashes were due to aircraft or weather
- Broad.phase.of.flight — E.g., takeoff, landing, cruise

#### ⚠️ Accident Details
- Injury.Severity — Fatal, serious, minor, none
- Aircraft.damage — Destroyed, substantial, minor
- Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, Total.Uninjured — Use these to calculate fatality ratio
- Purpose.of.flight — Distinguish between private, commercial, instructional use
- FAR.Description — Can indicate the regulation the flight was under (optional for advanced filtering)

## Data Preparation & Analysis

In [ ]:
df['Aircraft.Category'].value_counts()
df['Injury.Severity'].value_counts()

# Convert date to be sure of our data
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

df

Before performing any in-depth analysis, the data was cleaned and prepared to ensure accuracy. This involved handling missing values, standardizing aircraft names, and generating new features like fatality ratios. We then conducted exploratory analysis to identify trends, correlations, and outliers in accident rates across different aircraft types and usage categories. Visualizations supported this effort by highlighting key patterns in a clear and digestible format.

## Results & Recommendations

Based on the analysis, we developed three actionable business recommendations aimed at guiding safe and strategic investment in aircraft. Each recommendation is backed by empirical evidence from the dataset and aligns with operational considerations such as aircraft usage, maintenance needs, and historical safety performance.

### Business Recommendation 1

#### 1) Prioritize Aircraft Models with Proven Safety Records
Certain aircraft models consistently demonstrate lower accident and fatality rates, even in high-usage scenarios. Investing in these models can provide a strong balance between performance and safety.

### Business Recommendation 2

#### 2) Avoid Older Aircraft or Categories with High Risk
The data indicates a strong correlation between aircraft age and accident severity, particularly among older piston-engine aircraft. Avoiding high-risk categories during initial investment phases can reduce exposure to maintenance costs and safety liabilities.

### Business Recommendation 3

#### 3) Align Fleet Type with Purpose: Commercial Use = Jet/Turboprop; Private = Single-Engine Trainers
Risk levels differ by flight purpose and aircraft engine type. Aligning aircraft types with intended use—such as using turboprops for commercial routes and reliable single-engine planes for private use—can help optimize both safety and operational efficiency.

## Conclusion

The findings provide a data-driven foundation for selecting aircraft that align with your company’s safety standards and business objectives. By focusing on models with a strong track record, avoiding high-risk categories, and aligning aircraft types with use cases, your company can make informed decisions as it enters the aviation sector.

### Next Steps

Next steps include validating these insights with industry experts, incorporating financial and operational cost data, and exploring real-time aircraft performance metrics for continuous risk monitoring.